# 第 9 章：Seq2Seq 架构

> 序列到序列模型 - 机器翻译的基础架构

---

## 本章目标

- [ ] 理解编码器-解码器架构
- [ ] 实现基础 Seq2Seq 模型
- [ ] 理解 Teacher Forcing 训练策略
- [ ] 了解 Seq2Seq 的局限性

---

## Seq2Seq 概述

**应用场景**：
- 机器翻译：英文 → 中文
- 文本摘要：长文本 → 短摘要
- 对话系统：问题 → 回答
- 语音识别：音频 → 文本

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

---

## 第一部分：Seq2Seq 架构

### 1.1 编码器-解码器结构

```
编码器 (Encoder):
[x1] → [x2] → [x3] → [x4] → [context]
  "    I     love    you"      ↓
                               ↓
解码器 (Decoder):              ↓
         [context] → [y1] → [y2] → [y3]
                      "我"   "爱"   "你"
```

In [ ]:
class Encoder(nn.Module):
    """
    编码器：将输入序列编码为上下文向量
    """
    
    def __init__(self, vocab_size, embed_dim, hidden_size, num_layers=1, dropout=0.2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_size, num_layers, 
                           batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, src):
        """
        src: (batch, src_len)
        返回: hidden, cell (编码后的上下文)
        """
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.lstm(embedded)
        return hidden, cell


class Decoder(nn.Module):
    """
    解码器：根据上下文和之前的输出生成目标序列
    """
    
    def __init__(self, vocab_size, embed_dim, hidden_size, num_layers=1, dropout=0.2):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_size, num_layers,
                           batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, input, hidden, cell):
        """
        input: (batch, 1) 当前时间步的输入
        返回: output, hidden, cell
        """
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        prediction = self.fc(output.squeeze(1))
        return prediction, hidden, cell


class Seq2Seq(nn.Module):
    """
    完整的 Seq2Seq 模型
    """
    
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
    
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        """
        src: (batch, src_len) 源序列
        trg: (batch, trg_len) 目标序列
        """
        batch_size = src.size(0)
        trg_len = trg.size(1)
        trg_vocab_size = self.decoder.vocab_size
        
        # 存储输出
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)
        
        # 编码
        hidden, cell = self.encoder(src)
        
        # 第一个输入是 <sos>
        input = trg[:, 0:1]
        
        # 逐步解码
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[:, t, :] = output
            
            # Teacher Forcing
            teacher_force = np.random.random() < teacher_forcing_ratio
            top1 = output.argmax(1).unsqueeze(1)
            input = trg[:, t:t+1] if teacher_force else top1
        
        return outputs


# 创建模型
SRC_VOCAB = 1000
TRG_VOCAB = 1000
EMBED_DIM = 128
HIDDEN_SIZE = 256

encoder = Encoder(SRC_VOCAB, EMBED_DIM, HIDDEN_SIZE)
decoder = Decoder(TRG_VOCAB, EMBED_DIM, HIDDEN_SIZE)
model = Seq2Seq(encoder, decoder, device).to(device)

print(f"总参数量: {sum(p.numel() for p in model.parameters()):,}")

---

## 第二部分：信息瓶颈问题

### Seq2Seq 的局限性

所有源序列信息被压缩到一个固定大小的上下文向量中：

```
[很长的输入序列...] → [单个向量] → [输出序列]
                        ↑
                    信息瓶颈！
```

这个问题通过**注意力机制**解决（下一章）。

In [ ]:
def visualize_bottleneck():
    """
    可视化信息瓶颈问题
    """
    fig, ax = plt.subplots(figsize=(12, 5))
    
    # 编码器
    for i in range(5):
        rect = plt.Rectangle((i*1.5, 3), 1, 1, fill=True, 
                              facecolor='lightblue', edgecolor='blue')
        ax.add_patch(rect)
        ax.text(i*1.5 + 0.5, 3.5, f'$x_{i+1}$', ha='center', va='center')
    
    # 瓶颈
    circle = plt.Circle((8, 3.5), 0.5, fill=True, facecolor='red', alpha=0.7)
    ax.add_patch(circle)
    ax.text(8, 3.5, 'ctx', ha='center', va='center', color='white', fontweight='bold')
    
    # 解码器
    for i in range(4):
        rect = plt.Rectangle((9.5 + i*1.5, 3), 1, 1, fill=True,
                              facecolor='lightgreen', edgecolor='green')
        ax.add_patch(rect)
        ax.text(9.5 + i*1.5 + 0.5, 3.5, f'$y_{i+1}$', ha='center', va='center')
    
    # 箭头
    ax.annotate('', xy=(7.5, 3.5), xytext=(6.5, 3.5),
                arrowprops=dict(arrowstyle='->', lw=2))
    ax.annotate('', xy=(9.5, 3.5), xytext=(8.5, 3.5),
                arrowprops=dict(arrowstyle='->', lw=2))
    
    ax.set_xlim(-1, 16)
    ax.set_ylim(1, 6)
    ax.set_title('Seq2Seq 信息瓶颈：所有信息压缩到单个向量', fontsize=14)
    ax.text(8, 2, '瓶颈！', ha='center', fontsize=12, color='red')
    ax.axis('off')
    
    plt.show()


visualize_bottleneck()

---

## 本章总结

1. **编码器**：将输入序列编码为上下文向量
2. **解码器**：根据上下文生成输出序列
3. **Teacher Forcing**：训练时使用真实目标作为输入
4. **信息瓶颈**：固定大小的上下文向量限制了性能

---

## 下一章预告

**第 10 章：注意力机制**

👉 [10_attention_mechanism.ipynb](./10_attention_mechanism.ipynb)